In [17]:
import numpy as np

file = open('rosalind.txt','r')
Lines = file.readlines()
lines = []
seq = ""


for line in Lines:
    if line.find('>') == 0:
        lines.append(seq)
        seq = ""
    if line.find('>') != 0:
            seq = seq + line.strip('\n')

lines.append(seq)
lines.pop(0)

''

In [18]:
def BLOSUM62_scoring_matrix(s1,s2):
    sMatrixTxt = '''
   A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y
A  4  0 -2 -1 -2  0 -2 -1 -1 -1 -1 -2 -1 -1 -1  1  0  0 -3 -2
C  0  9 -3 -4 -2 -3 -3 -1 -3 -1 -1 -3 -3 -3 -3 -1 -1 -1 -2 -2
D -2 -3  6  2 -3 -1 -1 -3 -1 -4 -3  1 -1  0 -2  0 -1 -3 -4 -3
E -1 -4  2  5 -3 -2  0 -3  1 -3 -2  0 -1  2  0  0 -1 -2 -3 -2
F -2 -2 -3 -3  6 -3 -1  0 -3  0  0 -3 -4 -3 -3 -2 -2 -1  1  3
G  0 -3 -1 -2 -3  6 -2 -4 -2 -4 -3  0 -2 -2 -2  0 -2 -3 -2 -3
H -2 -3 -1  0 -1 -2  8 -3 -1 -3 -2  1 -2  0  0 -1 -2 -3 -2  2
I -1 -1 -3 -3  0 -4 -3  4 -3  2  1 -3 -3 -3 -3 -2 -1  3 -3 -1
K -1 -3 -1  1 -3 -2 -1 -3  5 -2 -1  0 -1  1  2  0 -1 -2 -3 -2
L -1 -1 -4 -3  0 -4 -3  2 -2  4  2 -3 -3 -2 -2 -2 -1  1 -2 -1
M -1 -1 -3 -2  0 -3 -2  1 -1  2  5 -2 -2  0 -1 -1 -1  1 -1 -1
N -2 -3  1  0 -3  0  1 -3  0 -3 -2  6 -2  0  0  1  0 -3 -4 -2
P -1 -3 -1 -1 -4 -2 -2 -3 -1 -3 -2 -2  7 -1 -2 -1 -1 -2 -4 -3
Q -1 -3  0  2 -3 -2  0 -3  1 -2  0  0 -1  5  1  0 -1 -2 -2 -1
R -1 -3 -2  0 -3 -2  0 -3  2 -2 -1  0 -2  1  5 -1 -1 -3 -3 -2
S  1 -1  0  0 -2  0 -1 -2  0 -2 -1  1 -1  0 -1  4  1 -2 -3 -2
T  0 -1 -1 -1 -2 -2 -2 -1 -1 -1 -1  0 -1 -1 -1  1  5  0 -2 -2
V  0 -1 -3 -2 -1 -3 -3  3 -2  1  1 -3 -2 -2 -3 -2  0  4 -3 -1
W -3 -2 -4 -3  1 -2 -2 -3 -3 -2 -1 -4 -4 -2 -3 -3 -2 -3 11  2
Y -2 -2 -3 -2  3 -3  2 -1 -2 -1 -1 -2 -3 -1 -2 -2 -2 -1  2  7
'''

    sMatrixList = sMatrixTxt.split('\n')
    sMatrixList.pop(0)
    del sMatrixList[-1]

    col = sMatrixList[0].index(s1)
    for i in range(len(sMatrixList)):
        if sMatrixList[i][0] == s2:
            score = int(sMatrixList[i][col-1]+sMatrixList[i][col])
    
    return score

In [44]:
def local_alignmen(lines):
    v = lines[0]
    w = lines[1]
    
    n = len(v)
    m = len(w)
    
    gap_penalty = 1
    gap_opening = 11
    
    
    #lower = np.ones((n+1,m+1), dtype= int)*(-1000000)
    #lower = np.ones((n+1,m+1), dtype= int)*(-1000000)
    #middle = np.ones((n+1,m+1), dtype= int)*(-1000000)
    #upper = np.ones((n+1,m+1), dtype= int)*(-1000000)
    lower = np.zeros((n+1,m+1), dtype= int)
    middle = np.zeros((n+1,m+1), dtype= int)
    upper = np.zeros((n+1,m+1), dtype= int)
    local_distance = [lower, middle, upper]
    
    direction_lower = np.zeros((n+1,m+1), dtype= int)   
    direction_middel = np.zeros((n+1,m+1), dtype= int)
    direction_upper = np.zeros((n+1,m+1), dtype= int)
    direction = [direction_lower, direction_middel, direction_upper]
    
    local_distance[0][0,0] = 0
    local_distance[1][0,0] = 0
    local_distance[0][1,0] = -gap_opening
    local_distance[1][1,0] = -gap_opening

    for i in np.arange(2,n+1):
        local_distance[0][i,0] = local_distance[0][i-1,0] - gap_penalty
        local_distance[1][i,0] = local_distance[0][i,0]
    
    
    
    local_distance[2][0,0] = 0
    local_distance[2][0,1] = -gap_opening
    local_distance[1][0,1] = -gap_opening
    
    for i in np.arange(2,m+1):
        local_distance[2][0,i] = local_distance[2][0,i-1] - gap_penalty
        local_distance[1][0,i] = local_distance[2][0,i]
    
    for i in range(1,n+1):
        for j in range(1,m+1):
            check_point = [local_distance[0][i-1,j]-gap_penalty,
                          local_distance[1][i-1,j]-gap_opening]
            max_score = max(check_point)
            local_distance[0][i,j] = max_score
            if check_point.index(max_score)==1:
                direction[0][i,j] = 1 
            
            
            check_point = [local_distance[2][i,j-1]-gap_penalty,
                          local_distance[1][i,j-1]-gap_opening]
            max_score = max(check_point)
            local_distance[2][i,j] = max_score
            if check_point.index(max_score)==1:
                direction[2][i,j] = 1
            
            
            match_score = BLOSUM62_scoring_matrix(v[i-1],w[j-1])
            check_point = [0, (local_distance[0][i,j]),
                           (local_distance[1][i-1,j-1]+match_score),
                           (local_distance[2][i,j])]
            max_score = max(check_point)
            local_distance[1][i,j] = max_score
            if max_score == check_point[0]:
                direction[1][i,j] = 3
            elif max_score == check_point[1]:
                direction[1][i,j] = 1
            elif max_score == check_point[2]:
                direction[1][i,j] = 2

    

    return local_distance[1][n,m],direction,local_distance

In [59]:
def bracktract(local_distance,direction,v,w):
    s1 = [] 
    s2 = []
    
    #indx1 = n
    #indx2 = m
    
    indx2 = np.argmax(np.max(local_distance[1], axis=0))
    print(indx2)
    indx1 = np.argmax(np.max(local_distance[1], axis=1))
    print(indx1)
    
    level = 1
    
    while indx1>0 or indx2>0:
        print('indx1'+' '+str(indx1))
        print('indx2'+' '+str(indx2))
        print('level'+' '+str(level))
        
        if level == 0:
            level = direction[0][indx1,indx2]
            indx1 -= 1
            s2.append('-')
            s1.append(v[indx1])
        
        elif level == 2:
            if direction[2][indx1,indx2]==1:
                level = 1
            indx2 -= 1
            s2.append(w[indx2])
            s1.append('-')
            
        elif level == 1:
            level = direction[1][indx1,indx2]
            if level == 1:
                indx1 -= 1
                indx2 -= 1
                s1.append(v[indx1])
                s2.append(w[indx2])
        elif level == 3:
            indx1 = 0
            indx2 = 0
                
    s1.reverse()
    s2.reverse()    
    
    return s1,s2


In [60]:
max_score, direction, local_distance = local_alignmen(lines)

v = lines[0]
w = lines[1]
s1, s2 = bracktract(local_distance,direction,v,w)

print(max_score)
x = "".join([str(i) for i in s1])
print(x)
x = "".join([str(i) for i in s2])
print(x)

6
10
indx1 10
indx2 6
level 1
indx1 10
indx2 6
level 2
indx1 10
indx2 5
level 2
indx1 10
indx2 4
level 2
indx1 10
indx2 3
level 2
indx1 10
indx2 2
level 2
indx1 10
indx2 1
level 2
indx1 10
indx2 0
level 2
indx1 10
indx2 -1
level 2
indx1 10
indx2 -2
level 2
indx1 10
indx2 -3
level 2
indx1 10
indx2 -4
level 2
indx1 10
indx2 -5
level 2
indx1 10
indx2 -6
level 2


IndexError: string index out of range

In [50]:
direction[0]

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1]])

In [51]:
direction[1]

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 3, 3, 3, 3, 3, 3],
       [0, 3, 3, 3, 3, 2, 3],
       [0, 3, 2, 3, 3, 3, 2],
       [0, 3, 3, 2, 3, 3, 3],
       [0, 3, 3, 2, 2, 3, 3],
       [0, 3, 3, 2, 3, 2, 3],
       [0, 3, 3, 3, 2, 3, 2],
       [0, 3, 3, 3, 3, 2, 3],
       [0, 3, 3, 3, 3, 2, 2],
       [0, 3, 3, 3, 3, 3, 2]])

In [61]:
direction[2]

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [47]:
local_distance[0]

array([[  0,   0,   0,   0,   0,   0,   0],
       [-11,  -1,  -1,  -1,  -1,  -1,  -1],
       [-12,  -2,  -2,  -2,  -2,  -2,  -2],
       [-13,  -3,  -3,  -3,  -3,  -3,  -3],
       [-14,  -4,  -4,  -4,  -4,  -4,  -4],
       [-15,  -5,  -5,  -2,  -5,  -5,  -5],
       [-16,  -6,  -6,  -3,  -1,  -6,  -6],
       [-17,  -7,  -7,  -4,  -2,  -2,  -7],
       [-18,  -8,  -8,  -5,  -1,  -3,  -4],
       [-19,  -9,  -9,  -6,  -2,  -2,  -5],
       [-20, -10, -10,  -7,  -3,  -3,  -3]])

In [48]:
local_distance[1]

array([[  0, -11, -12, -13, -14, -15, -16],
       [-11,   0,   0,   0,   0,   0,   0],
       [-12,   0,   0,   0,   0,   4,   0],
       [-13,   0,   5,   0,   0,   0,   2],
       [-14,   0,   0,   9,   0,   0,   0],
       [-15,   0,   0,   1,  10,   0,   0],
       [-16,   0,   0,   4,   0,   9,   0],
       [-17,   0,   0,   0,  10,   0,   7],
       [-18,   0,   0,   0,   0,   9,   0],
       [-19,   0,   0,   0,   0,   4,   8],
       [-20,   0,   0,   0,   0,   0,  11]])

In [49]:
local_distance[2]

array([[  0, -11, -12, -13, -14, -15, -16],
       [  0,  -1,  -2,  -3,  -4,  -5,  -6],
       [  0,  -1,  -2,  -3,  -4,  -5,  -6],
       [  0,  -1,  -2,  -3,  -4,  -5,  -6],
       [  0,  -1,  -2,  -3,  -2,  -3,  -4],
       [  0,  -1,  -2,  -3,  -4,  -1,  -2],
       [  0,  -1,  -2,  -3,  -4,  -5,  -2],
       [  0,  -1,  -2,  -3,  -4,  -1,  -2],
       [  0,  -1,  -2,  -3,  -4,  -5,  -2],
       [  0,  -1,  -2,  -3,  -4,  -5,  -6],
       [  0,  -1,  -2,  -3,  -4,  -5,  -6]])